# Here we talk about how we preprocessing data and imputation.
## Part 3 of EDA

I have a uncleaned dataset. Firstly I clean the data, combing some attributes and extract some other attributes from the existed attributes, with column transformation. I did not do any changes with the rows. Secondly, I split the data to train and test set, and use imputation method on train sets, then do the same thing on the test set. Thirdly, I use train set to train my Ml model and use test set to test the performance.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

In [7]:
data = pd.read_csv('data/EDA_data.csv')
data

,days_to_depart,flight_day_of_week,startingAirport,destinationAirport,departure_time,arriving_time,isDeparture_time_6to18,isArriving_time_11to24,stops,route,...,travelDuration_h,elapsedDays,isBasicEconomy,isRefundable,isNonStop,seatsRemaining,totalTravelDistance,total_waiting_time,baseFare,totalFare
0,44,3,LAX,ORD,12:55:00,00:54:00,True,False,1,LAX->DFW->ORD,...,9.983333,1,False,False,False,0,NaN,4.37,21.11,91.29
1,44,3,LAX,ORD,15:21:00,21:24:00,True,True,0,LAX->ORD,...,4.050000,0,False,False,True,0,NaN,0.00,43.00,99.89
2,44,3,LAX,ORD,07:00:00,13:04:00,True,True,0,LAX->ORD,...,4.066667,0,False,False,True,0,NaN,0.00,43.00,99.89
3,44,3,LAX,ORD,20:25:00,04:13:00,False,False,1,LAX->LAS->ORD,...,5.800000,1,False,False,False,0,NaN,0.92,33.00,103.18
4,44,3,LAX,ORD,19:25:00,04:13:00,False,False,1,LAX->LAS->ORD,...,6.800000,1,False,False,False,0,NaN,1.92,33.00,103.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88784,53,5,ATL,ORD,11:18:00,16:05:00,True,True,1,ATL->DCA->ORD,...,5.783333,0,False,False,False,7,1135.0,1.67,175.81,213.20
88785,53,5,ATL,ORD,12:24:00,17:15:00,True,True,1,ATL->DFW->ORD,...,5.850000,0,False,False,False,7,1524.0,1.02,175.81,213.20
88786,53,5,ATL,ORD,11:42:00,17:49:00,True,True,1,ATL->JFK->ORD,...,7.116667,0,False,False,False,7,1492.0,1.85,202.79,242.20
88787,53,5,ATL,ORD,17:45:00,18:59:00,True,True,0,ATL->ORD,...,2.233333,0,False,False,True,9,600.0,0.00,226.98,258.90


In [8]:
data.isna().sum()

days_to_depart                0
flight_day_of_week            0
startingAirport               0
destinationAirport            0
departure_time                0
arriving_time                 0
isDeparture_time_6to18        0
isArriving_time_11to24        0
stops                         0
route                         0
class                         0
airline                       0
travelDuration_h              0
elapsedDays                   0
isBasicEconomy                0
isRefundable                  0
isNonStop                     0
seatsRemaining                0
totalTravelDistance       10584
total_waiting_time            0
baseFare                      0
totalFare                     0
dtype: int64

Add an indicator to this missing attributes

In [9]:
data['isTravel_dist_available'] = -data['totalTravelDistance'].isna()
data

,days_to_depart,flight_day_of_week,startingAirport,destinationAirport,departure_time,arriving_time,isDeparture_time_6to18,isArriving_time_11to24,stops,route,...,elapsedDays,isBasicEconomy,isRefundable,isNonStop,seatsRemaining,totalTravelDistance,total_waiting_time,baseFare,totalFare,isTravel_dist_available
0,44,3,LAX,ORD,12:55:00,00:54:00,True,False,1,LAX->DFW->ORD,...,1,False,False,False,0,NaN,4.37,21.11,91.29,False
1,44,3,LAX,ORD,15:21:00,21:24:00,True,True,0,LAX->ORD,...,0,False,False,True,0,NaN,0.00,43.00,99.89,False
2,44,3,LAX,ORD,07:00:00,13:04:00,True,True,0,LAX->ORD,...,0,False,False,True,0,NaN,0.00,43.00,99.89,False
3,44,3,LAX,ORD,20:25:00,04:13:00,False,False,1,LAX->LAS->ORD,...,1,False,False,False,0,NaN,0.92,33.00,103.18,False
4,44,3,LAX,ORD,19:25:00,04:13:00,False,False,1,LAX->LAS->ORD,...,1,False,False,False,0,NaN,1.92,33.00,103.18,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88784,53,5,ATL,ORD,11:18:00,16:05:00,True,True,1,ATL->DCA->ORD,...,0,False,False,False,7,1135.0,1.67,175.81,213.20,True
88785,53,5,ATL,ORD,12:24:00,17:15:00,True,True,1,ATL->DFW->ORD,...,0,False,False,False,7,1524.0,1.02,175.81,213.20,True
88786,53,5,ATL,ORD,11:42:00,17:49:00,True,True,1,ATL->JFK->ORD,...,0,False,False,False,7,1492.0,1.85,202.79,242.20,True
88787,53,5,ATL,ORD,17:45:00,18:59:00,True,True,0,ATL->ORD,...,0,False,False,True,9,600.0,0.00,226.98,258.90,True


Based on EDA, we shall drop the following variables

In [10]:
data = data.drop('departure_time',axis=1)
data = data.drop('arriving_time',axis=1)
data = data.drop('isRefundable',axis=1)

### Transform data

Transform categorical data to dummmies. By one-hot encoder

*Noticed that we need to do this before split train and test, since it may cause different features between train and test if doing after splitting.*

In [11]:
def transform_cate_data(df1, ignore_route=True):
    df = df1.copy()
    bool_cols = df.select_dtypes(include=['bool']).columns.tolist()
    df[bool_cols] = df[bool_cols].astype(float)
    cat_cols = df.select_dtypes(include=['object']).columns.tolist()
    if ignore_route == True:
        cat_cols = list(set(cat_cols) - {'route'})
    ohe = OneHotEncoder(sparse_output=False)
    cat_transformed = ohe.fit_transform(df[cat_cols])
    df_transformed = pd.concat(
        [df.drop(cat_cols, axis=1), pd.DataFrame(cat_transformed, columns=ohe.get_feature_names_out(cat_cols))], axis=1)
    return df_transformed

In [12]:
data = transform_cate_data(data, ignore_route=True)

In [13]:
data

,days_to_depart,flight_day_of_week,isDeparture_time_6to18,isArriving_time_11to24,stops,route,travelDuration_h,elapsedDays,isBasicEconomy,isNonStop,...,class_first,class_first+coach,class_first+coach+first,class_premium coach,class_premium coach+coach,destinationAirport_ATL,destinationAirport_DFW,destinationAirport_JFK,destinationAirport_LAX,destinationAirport_ORD
0,44,3,1.0,0.0,1,LAX->DFW->ORD,9.983333,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,44,3,1.0,1.0,0,LAX->ORD,4.050000,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,44,3,1.0,1.0,0,LAX->ORD,4.066667,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,44,3,0.0,0.0,1,LAX->LAS->ORD,5.800000,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,44,3,0.0,0.0,1,LAX->LAS->ORD,6.800000,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88784,53,5,1.0,1.0,1,ATL->DCA->ORD,5.783333,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
88785,53,5,1.0,1.0,1,ATL->DFW->ORD,5.850000,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
88786,53,5,1.0,1.0,1,ATL->JFK->ORD,7.116667,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
88787,53,5,1.0,1.0,0,ATL->ORD,2.233333,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### Split data into train and test set

In [14]:
# Split data into training and test sets
train_data = data.sample(frac=0.8, random_state=41)
test_data = data.drop(train_data.index)

In [15]:
train_data

,days_to_depart,flight_day_of_week,isDeparture_time_6to18,isArriving_time_11to24,stops,route,travelDuration_h,elapsedDays,isBasicEconomy,isNonStop,...,class_first,class_first+coach,class_first+coach+first,class_premium coach,class_premium coach+coach,destinationAirport_ATL,destinationAirport_DFW,destinationAirport_JFK,destinationAirport_LAX,destinationAirport_ORD
36737,39,5,1.0,1.0,1,ORD->CLT->LAX,8.233333,0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
37172,21,1,1.0,1.0,0,LAX->DFW,3.100000,0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
13133,14,1,1.0,0.0,0,DFW->ORD,2.316667,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
38121,21,1,1.0,1.0,2,JFK->RDU->IAH->DFW,10.150000,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
20022,27,0,1.0,0.0,0,LAX->JFK,5.416667,1,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19743,27,0,1.0,1.0,0,JFK->ATL,2.566667,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
43797,51,3,1.0,1.0,1,LAX->ORD->ATL,8.116667,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
73695,23,3,1.0,1.0,1,JFK->PIT->DFW,7.133333,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
73941,23,3,1.0,1.0,1,DFW->SGF->ORD,4.566667,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [16]:
Y_train = train_data[['baseFare', 'totalFare']]
X_train = train_data.drop(['baseFare', 'totalFare'], axis=1)
Y_test = test_data[['baseFare', 'totalFare']]
X_test = test_data.drop(['baseFare', 'totalFare'], axis=1)

### Dealing with NA data

Here we impute by using the avg dist of same route.

In [17]:
def fill_travel_distance(x, dict):
    if x in list(dict.keys()):
        value = dict[x]
    else:
        value = np.NAN
    return value


def fill_travel_dist_by_dict(traindf, testdf):
    # fillNA by same route, avg distance, from train data
    train = traindf.copy()
    test = testdf.copy()
    helper1 = train[-train['totalTravelDistance'].isna()][['totalTravelDistance', 'route']].groupby('route').mean()
    helper1_dict = helper1.to_dict()['totalTravelDistance']
    train['totalTravelDistance_helper'] = train['route'].apply(lambda x: fill_travel_distance(x, helper1_dict))
    train['totalTravelDistance'] = train['totalTravelDistance_helper']
    train = train.drop('totalTravelDistance_helper', axis=1)
    test['totalTravelDistance_helper'] = test['route'].apply(lambda x: fill_travel_distance(x, helper1_dict))
    test['totalTravelDistance'] = test['totalTravelDistance_helper']
    test = test.drop('totalTravelDistance_helper', axis=1)
    return train, test

In [18]:
X_train1, X_test1 = fill_travel_dist_by_dict(X_train, X_test)

### Then we apply cate transformer and use KNN to impute the missing value

In [19]:
X_train1 = X_train1.reset_index(drop=True)
X_test1 = X_test1.reset_index(drop=True)
X_train1

,days_to_depart,flight_day_of_week,isDeparture_time_6to18,isArriving_time_11to24,stops,route,travelDuration_h,elapsedDays,isBasicEconomy,isNonStop,...,class_first,class_first+coach,class_first+coach+first,class_premium coach,class_premium coach+coach,destinationAirport_ATL,destinationAirport_DFW,destinationAirport_JFK,destinationAirport_LAX,destinationAirport_ORD
0,39,5,1.0,1.0,1,ORD->CLT->LAX,8.233333,0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,21,1,1.0,1.0,0,LAX->DFW,3.100000,0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,14,1,1.0,0.0,0,DFW->ORD,2.316667,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,21,1,1.0,1.0,2,JFK->RDU->IAH->DFW,10.150000,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,27,0,1.0,0.0,0,LAX->JFK,5.416667,1,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71026,27,0,1.0,1.0,0,JFK->ATL,2.566667,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
71027,51,3,1.0,1.0,1,LAX->ORD->ATL,8.116667,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
71028,23,3,1.0,1.0,1,JFK->PIT->DFW,7.133333,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
71029,23,3,1.0,1.0,1,DFW->SGF->ORD,4.566667,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [20]:
X_train2 = X_train1.drop('route', axis=1)
X_test2 = X_test1.drop('route', axis=1)

In [21]:
from sklearn.impute import KNNImputer

In [22]:
# imputer = IterativeImputer(estimator=LinearRegression())
imputer = KNNImputer(n_neighbors=5)
X_train_imputed = imputer.fit_transform(X_train2)
X_test_imputed = imputer.transform(X_test2)
X_train_imputed = pd.DataFrame(X_train_imputed, columns=X_train2.columns)
X_test_imputed = pd.DataFrame(X_test_imputed, columns=X_test2.columns)

In [23]:
X_train_imputed

,days_to_depart,flight_day_of_week,isDeparture_time_6to18,isArriving_time_11to24,stops,travelDuration_h,elapsedDays,isBasicEconomy,isNonStop,seatsRemaining,...,class_first,class_first+coach,class_first+coach+first,class_premium coach,class_premium coach+coach,destinationAirport_ATL,destinationAirport_DFW,destinationAirport_JFK,destinationAirport_LAX,destinationAirport_ORD
0,39.0,5.0,1.0,1.0,1.0,8.233333,0.0,1.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,21.0,1.0,1.0,1.0,0.0,3.100000,0.0,1.0,1.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,14.0,1.0,1.0,0.0,0.0,2.316667,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,21.0,1.0,1.0,1.0,2.0,10.150000,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,27.0,0.0,1.0,0.0,0.0,5.416667,1.0,0.0,1.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71026,27.0,0.0,1.0,1.0,0.0,2.566667,0.0,0.0,1.0,7.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
71027,51.0,3.0,1.0,1.0,1.0,8.116667,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
71028,23.0,3.0,1.0,1.0,1.0,7.133333,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
71029,23.0,3.0,1.0,1.0,1.0,4.566667,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [24]:
X_train_imputed.isna().sum()

days_to_depart                     0
flight_day_of_week                 0
isDeparture_time_6to18             0
isArriving_time_11to24             0
stops                              0
travelDuration_h                   0
elapsedDays                        0
isBasicEconomy                     0
isNonStop                          0
seatsRemaining                     0
totalTravelDistance                0
total_waiting_time                 0
isTravel_dist_available            0
airline_9K+UA                      0
airline_9X+AA                      0
airline_AA                         0
airline_AA+9X                      0
airline_AA+LF                      0
airline_AS                         0
airline_AS+DL                      0
airline_AS+UA                      0
airline_B6                         0
airline_DL                         0
airline_DL+AS                      0
airline_DL+UA                      0
airline_F9                         0
airline_NK                         0
a

In [25]:
Y_train = Y_train.reset_index(drop=True)
Y_test = Y_test.reset_index(drop=True)

In [26]:
total_train = pd.concat([X_train_imputed,Y_train],axis=1)
total_test = pd.concat([X_test_imputed,Y_test],axis=1)

In [27]:
total_train

,days_to_depart,flight_day_of_week,isDeparture_time_6to18,isArriving_time_11to24,stops,travelDuration_h,elapsedDays,isBasicEconomy,isNonStop,seatsRemaining,...,class_first+coach+first,class_premium coach,class_premium coach+coach,destinationAirport_ATL,destinationAirport_DFW,destinationAirport_JFK,destinationAirport_LAX,destinationAirport_ORD,baseFare,totalFare
0,39.0,5.0,1.0,1.0,1.0,8.233333,0.0,1.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,143.26,176.71
1,21.0,1.0,1.0,1.0,0.0,3.100000,0.0,1.0,1.0,7.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,124.65,148.90
2,14.0,1.0,1.0,0.0,0.0,2.316667,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,56.00,112.89
3,21.0,1.0,1.0,1.0,2.0,10.150000,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,486.52,552.01
4,27.0,0.0,1.0,0.0,0.0,5.416667,1.0,0.0,1.0,9.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,487.44,538.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71026,27.0,0.0,1.0,1.0,0.0,2.566667,0.0,0.0,1.0,7.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,73.49,93.90
71027,51.0,3.0,1.0,1.0,1.0,8.116667,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,358.14,409.20
71028,23.0,3.0,1.0,1.0,1.0,7.133333,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,232.56,274.20
71029,23.0,3.0,1.0,1.0,1.0,4.566667,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,254.88,298.20


In [28]:
total_test

,days_to_depart,flight_day_of_week,isDeparture_time_6to18,isArriving_time_11to24,stops,travelDuration_h,elapsedDays,isBasicEconomy,isNonStop,seatsRemaining,...,class_first+coach+first,class_premium coach,class_premium coach+coach,destinationAirport_ATL,destinationAirport_DFW,destinationAirport_JFK,destinationAirport_LAX,destinationAirport_ORD,baseFare,totalFare
0,44.0,3.0,0.0,0.0,1.0,6.383333,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,56.11,126.29
1,44.0,3.0,1.0,0.0,1.0,13.133333,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,67.00,137.18
2,44.0,3.0,0.0,0.0,1.0,6.133333,1.0,1.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,143.26,178.21
3,44.0,3.0,1.0,0.0,1.0,11.666667,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,73.00,181.18
4,44.0,3.0,1.0,1.0,0.0,4.083333,0.0,0.0,1.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,191.63,220.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17753,53.0,5.0,1.0,1.0,1.0,15.566667,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11.00,119.18
17754,53.0,5.0,1.0,1.0,1.0,5.600000,0.0,1.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,132.09,166.20
17755,53.0,5.0,1.0,1.0,1.0,5.850000,0.0,1.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,143.26,178.21
17756,53.0,5.0,0.0,1.0,0.0,2.066667,0.0,0.0,1.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,175.81,203.90


In [30]:
# total_train.to_csv('data/train.csv',index=False)
# total_test.to_csv('data/test.csv',index=False)

Based on the feature importance and data distribution, we also need to reconsider the features. See Modeling part1 for more details on discussion.